# Non-Localized Gravitational Wave Distances

The non localized distance to a gravitaitonal wave signal can be estimated using the equation 

$$D = \frac{2c}{h_+} \left( \frac{G \mathcal{M}}{c^3}\right)^{5/3} \Omega^{\frac{2}{3}}(t) \left(1+cos^2i\right) cos2\Phi(t)$$

where $h_+$ is the maximum strain of the GW, $\mathcal{M}$ is the Chirp Mass of the gravitational wave. $\Omega(t)$ is the Keplerian orbital frequency over time. "Keplerian" just means it was derived from Kepler's laws of orbital motion and Newton's theory of gravity instead of Einsteins theory of General Relativity. $\Phi(t)$ is the accumulated orbital phase (this term is a type of mathemitcal correction to fix the build up of errors when integrating the orbital frequency $\Omega$ over a continuous time interval). 


Note: When perfoming the calculation, however, you get a more accurate estimate of the distance using the Combined mass $M$ the GW instead of the Chirp mass $\mathcal{M}$. I am not sure why this is the case, but I will keep looking. If you have any ideas as to why this discrepency exist, let me know!



The goal of this assignment is to derive an analytic expression for $D$  which can be used to estimate distances. To do this, we first need to derive a funcitonal form of  $\Omega(t)$  and  $\Phi(t)$. 

The first equation you will be deriving is $\Omega(t)$. To do this, solve the first order differential equation below for an expression where $\Omega \propto t$. Watch your exponents! 


$$\frac{d\Omega}{dt} = \frac{96}{5}\left(\frac{G \mathcal{M}}{c^3}\right)^{5/3}\Omega^{11/3}$$

Next, we will need to derive an expression for the function $\Phi(t)$. The general equation for $\Phi(t)$ is:

$$\Phi(t) = \int_t{\Omega(t)} dt$$

To solve for $\Phi(t)$, plug your expression for $\omega(t)$ into the integral above and integrate from $0 \rightarrow t$, and once again, watch your exponents!

And that should be all the parts we need to begin developing a calculator! If you are feeling adventurous, try plugging in these values from GW150914. The observable parameters you need are $Combined$ $Mass$ ($M$), the maximum strain ($h_+$), and the duration of the signal in seconds ($t$). 

$t = 32s$

$M = 63.1 M_{\odot}$

$h_+ = 7.912043421880075 \times 10^{-19} Hz$

In [6]:
import math
import numpy as np
from astropy.table import Table

In [7]:
G = 6.67e-11 # N kg^-2 m^2
c = 3e8 # m/s

def distance_to_GW(t, M_sol, h_max, i):
    '''This function will give the non-localized distance (in Mpc) to the gravitational wave 
    when inputting time in seconds, combined mass (or chirp mass) in solar masses, and the maximum strain in Hz.'''
    
    M = M_sol * (2 * 10**30) # gives mass in kg
    
    term1 = (5/256)**(3/8)
    term2 = ((c**3) / (G * M))**(5/8)
    term3 = (1 / (t**(3/8)))
    term4 = (t**(5/8))
    
    orbital_freq = term1 * term2 * term3
    orbital_phase = np.round(0.36571582 * term2 * term4)
    
    distance = (2*c / h_max) * (G*M / (c**3))**(5/3) * orbital_freq**(2/3) *(1 + np.cos(i)**2)* abs(np.cos(2 * orbital_phase)) # this is distance in meters
    
    #print(orbital_freq) # printed this just to check the value of it
    #print(orbital_phase) # printed this just to check the value of it
    
    return distance / (9.223 * 10**18)#i returns distance in Mpc. 2.25 update: change [distance / (9.223 * 10**18), i]
    # distance only, for debugging best_i_arr. If anyone need to use best_angle again, please change the return statement to the original one

distance_to_GW(32, 63.1, 7.912043421880075e-19, np.pi/2)

394.17565185908205

In [8]:
array_of_inclinations = np.array([distance_to_GW(32, 63.1, 7.912043421880075e-19, x) for x in np.arange(0, np.pi/2, np.pi/180)])

In [9]:
def best_i(actual_dis, f): # a helper function for best_i_arr, does similar things as best_angle
    angle_lst = np.arange(0, math.pi / 2, math.pi / 180)
    best_dif = float('inf')
    best_i = -1
    for i in range(len(angle_lst)):
        d = f(angle_lst[i])
        if abs(d - actual_dis) < best_dif:
            best_i = angle_lst[i]
            best_dif = abs(d - actual_dis)

    return best_i

def best_i_arr(actual_arr, t_arr, chirp_mass_arr, h_max_arr): # give it arrays of: actual distance, time, h_max, and chirp_mass
    i_array = []
    f_arr = [] # a helper function array that returns gw_distance but only need to input angle i
    for n in range(len(t_arr)):
        f_arr.append(lambda x: distance_to_GW(t_arr[n],chirp_mass_arr[n], h_max_arr[n], x))

    for m in range(len(f_arr)):
        i_array.append(best_i(actual_arr[m], f_arr[m]))
        print(best_i(actual_arr[m], f_arr[m]))

    return i_array

def gw_dis_array(t_arr, chirp_mass_arr, h_max_arr, i_arr):
    ret = []
    for i in range(len(t_arr)):
        ret.append(distance_to_GW(t_arr[i], chirp_mass_arr[i], h_max_arr[i], i_arr[i]))
    return ret


In [45]:
best_i(440, lambda i: distance_to_GW(32, 63.1, 7.912043421880075e-19,i))

1.2217304763960306

In [46]:
best_i_arr([440], [32], [63.1], [7.912043421880075e-19])

1.2217304763960306


[1.2217304763960306]

In [47]:
gw_dis_array([32], [63.1], [7.912043421880075e-19], [math.pi/2])

[394.175651859082]

All the data:

In [ ]:
GW150914 = 'H-H1_GWOSC_16KHZ_R1-1126259447-32.txt'
GW151226 = 'H-H1_GWOSC_16KHZ_R1-1135136335-32.txt'
GW170104 = 'H-H1_GWOSC_16KHZ_R1-1167559921-32.txt'
GW170608 = 'H-H1_GWOSC_16KHZ_R1-1180922479-32.txt'
GW170814 = 'H-H1_GWOSC_16KHZ_R1-1186741846-32.txt'
GW170817 = 'H-H1_GWOSC_16KHZ_R1-1187008867-32.txt'
GW190425 = 'L-L1_GWOSC_16KHZ_R1-1240215487-32.txt' #not sure about this one, there was no H1 file for this, only L1 and V1
GW190412 = 'H-H1_GWOSC_16KHZ_R1-1239082247-32.txt'
GW190814 = 'H-H1_GWOSC_16KHZ_R1-1249852241-32.txt'
GW190521 = 'H-H1_GWOSC_16KHZ_R2-1242442952-32.txt'

GW150914_tbl = Table.read(GW150914, format='ascii')
GW151226_tbl = Table.read(GW151226, format='ascii')
GW170104_tbl = Table.read(GW170104, format='ascii')
GW170608_tbl = Table.read(GW170608, format='ascii')
GW170814_tbl = Table.read(GW170814, format='ascii')
GW170817_tbl = Table.read(GW170817, format='ascii')
GW190425_tbl = Table.read(GW190425, format='ascii') #this is the L1 file
GW190412_tbl = Table.read(GW190412, format='ascii')
GW190814_tbl = Table.read(GW190814, format='ascii')
GW190521_tbl = Table.read(GW190521, format='ascii')

In [ ]:
GW150914_strain = GW150914_tbl['6.0951345581611108e-21']
GW151226_strain = GW151226_tbl['2.7996600287459531e-20']
GW170104_strain = GW170104_tbl['-2.9550085672987675e-19']
GW170608_strain = GW170608_tbl['1.6421373582598675e-20']
GW170814_strain = GW170814_tbl['-7.6681619209110251e-19']
GW170817_strain = GW170817_tbl['5.8751133791669278e-19']
GW190425_strain = GW190425_tbl['-5.5244482372502713e-20'] #this is the L1 file
GW190412_strain = GW190412_tbl['-4.4229543869287918e-19']
GW190814_strain = GW190814_tbl['-3.0160263673448252e-20']
GW190521_strain = GW190521_tbl['-6.9973189495553901e-21']

In [ ]:
np.max(GW150914_strain)

In [ ]:
luminosity_dist = np.array[440, 450, 990, 320, 600, 40, 159, 740, 241, 5300]